# SfM

In [2]:
import os

%load_ext autoreload
%autoreload 2

src_path = os.getcwd()
print(src_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/unbinilium/Documents/Develop/Python/SfM/src


## Common Configs

In [3]:
datasets_path = '../data/datasets'
datasets_name = 'vegetable_dog_full_masked_sp_ex_sg'
use_masked_images = True
mvs_path = '/usr/local/bin/OpenMVS'

image_dir_path = os.path.join(src_path, datasets_path, datasets_name, 'images')
print(image_dir_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/vegetable_dog_full_masked_sp_ex_sg/images


## Generate Masks

In [4]:
import core.generate_masked_images

In [5]:
hypar_conf = {
    'model_path': '../data/models/is-net',  # load trained weights from this path
    'restore_model': 'isnet.pth',           # name of the to-be-loaded weights
    'interm_sup': False,                    # indicate if activate intermediate feature supervision
    'model_digit': 'full',                  # indicates 'half' or 'full' accuracy of float number
    'seed': 0,
    'cache_size': [1024, 1024],             # cached input spatial resolution, can be configured into different size
    'input_size': [1024, 1024],             # mdoel input spatial size, usually use the same value hypar['cache_size'], which means we don't further resize the images
    'crop_size': [1024, 1024],              # random crop size from the input, it is usually set as smaller than hypar['cache_size'], e.g., [920,920] for data augmentation
}

image_masks_path = os.path.join(src_path, datasets_path, datasets_name, 'masks')
masked_image_path = os.path.join(src_path, datasets_path, datasets_name, 'masked_images')
print(image_masks_path, masked_image_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/vegetable_dog_full_masked_sp_ex_sg/masks /Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/vegetable_dog_full_masked_sp_ex_sg/masked_images


In [7]:
core.generate_masked_images.main(
    image_dir_path,
    image_masks_path,
    masked_image_path,
    hypar_conf)

Old masks out directory exist: /Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/vegetable_dog_full_masked_sp_ex_sg/masks
Old masked images out directory exist: /Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/vegetable_dog_full_masked_sp_ex_sg/masked_images
Initializing DIS Network...
Running inference on device "cpu"
Generating masks and masked images...


100%|██████████| 107/107 [04:32<00:00,  2.55s/it]


## Extract Features

In [ ]:
import core.extract_features

In [ ]:
superpoint_conf = {
    'output': 'feats-spp',
    'model': {
        'path': '../thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth',
        'name': 'spp_det'
    },
    'preprocessing': {
        'grayscale': True
    },
    'conf': {
        'descriptor_dim': 256,
        'nms_radius': 3,
        'max_keypoints': 4096,
        'keypoints_threshold': 0.6
    }
}

features_out_path = os.path.join(src_path, datasets_path, datasets_name, superpoint_conf['output'] + '.h5')

print(features_out_path)

In [ ]:
core.extract_features.main(
    image_dir_path if not use_masked_images else masked_image_path,
    features_out_path,
    superpoint_conf)

## Generate Pairs

In [ ]:
import core.generate_pairs

In [ ]:
generate_pairs_conf = {
    'method': 'exhaustive',
    'num_matched': 3,
    'min_rotation': 10,
    'feature_path': features_out_path,
    'superglue': {
        'model': {
            'path': '../thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_outdoor.pth'
        },
        'output': 'matches-spg',
        'conf': {
            'descriptor_dim': 256,
            'weights': 'outdoor',
            'match_threshold': 0.7
        }
    }
}

pairs_out_path = os.path.join(src_path, datasets_path, datasets_name, '{}-pairs.txt'.format(generate_pairs_conf['method'])) 
print(pairs_out_path)

In [ ]:
core.generate_pairs.main(
    image_dir_path if not use_masked_images else masked_image_path,
    pairs_out_path,
    generate_pairs_conf)

## Matching Features

In [ ]:
import core.match_features

In [ ]:
match_features_conf = {
    'model': {
        'path': '../thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_indoor.pth'
    },
    'output': 'matches-spg',
    'conf': {
        'descriptor_dim': 256,
        'weights': 'indoor',
        'match_threshold': 0.7
    }
}

matches_out_path = os.path.join(src_path, datasets_path, datasets_name, '{}.h5'.format(match_features_conf['output']))
print(matches_out_path)

In [ ]:
core.match_features.main(
    features_out_path,
    pairs_out_path,
    matches_out_path,
    match_features_conf)

## Generate Empty

In [ ]:
import core.generate_empty

In [ ]:
sfm_empty_path = os.path.join(src_path, datasets_path, datasets_name, 'sfm_empty')
print(sfm_empty_path)

In [ ]:
core.generate_empty.main(
    image_dir_path if not use_masked_images else masked_image_path,
    sfm_empty_path)

## Triangulation

In [ ]:
import core.triangulation

In [ ]:
deep_sfm_path = os.path.join(src_path, datasets_path, datasets_name, 'sfm_ws')
sfm_outputs_path = os.path.join(deep_sfm_path, 'outputs')

print(deep_sfm_path, sfm_outputs_path)

In [ ]:
core.triangulation.main(
    deep_sfm_path,
    sfm_empty_path,
    sfm_outputs_path,
    pairs_out_path,
    features_out_path,
    matches_out_path,
    image_dir=image_dir_path if not use_masked_images else masked_image_path)

## Convert Workspace

In [ ]:
mvs_ws_path = os.path.join(src_path, datasets_path, datasets_name, 'mvs_ws')
mvs_images_path = os.path.join(src_path, datasets_path, datasets_name, 'mvs_ws/images')

if os.path.exists(mvs_ws_path):
    print('Old MVS workspace exist, removing...')
    cmd = ' '.join(['rm', '-fr', mvs_ws_path])
    os.system(cmd)
os.mkdir(mvs_ws_path)

print(mvs_ws_path, mvs_images_path)

In [ ]:
file_cmds = [' '.join([
    'cp', '-r',
    os.path.join(deep_sfm_path, 'model'),
    os.path.join(deep_sfm_path, 'sparse')
]), ' '.join([
    'ln', '-s',
    image_dir_path if not use_masked_images else masked_image_path,
    mvs_images_path
])]

mvs_cmd = ' '.join([
    os.path.join(mvs_path,'InterfaceCOLMAP'),
    '--working-folder', mvs_ws_path,
    '--input-file', deep_sfm_path,
    '--output-file', os.path.join(mvs_ws_path, 'model/model_colmap.mvs')
])

print(file_cmds, '\n', mvs_cmd)

In [ ]:
[os.system(c) for c in file_cmds]

In [ ]:
os.system(mvs_cmd)

## Densify Point Cloud

In [ ]:
mvs_cmd = ' '.join([
    os.path.join(mvs_path,'DensifyPointCloud'),
    '--archive-type', '-1',
    '--working-folder', mvs_ws_path,
    '--input-file', os.path.join(mvs_ws_path, 'model/model_colmap.mvs'),
    '--output-file', os.path.join(mvs_ws_path, 'model/model_dense.mvs'),
])
print(mvs_cmd)

In [ ]:
os.system(mvs_cmd)

## Reconstruct Mesh

In [ ]:
mvs_cmd = ' '.join([
    os.path.join(mvs_path, 'ReconstructMesh'),
    '--working-folder', mvs_ws_path,
    '--input-file', os.path.join(mvs_ws_path, 'model/model_dense.mvs'),
    '--output-file', os.path.join(mvs_ws_path, 'model/model_dense_mesh.mvs')
])
print(mvs_cmd)

In [ ]:
os.system(mvs_cmd)

## Refine Mesh

In [ ]:
mvs_cmd = ' '.join([
    os.path.join(mvs_path, 'RefineMesh'),
    '--resolution-level', '1',
    '--working-folder', mvs_ws_path,
    '--input-file', os.path.join(mvs_ws_path, 'model/model_dense_mesh.mvs'),
    '--output-file', os.path.join(mvs_ws_path, 'model/model_dense_mesh_refine.mvs')
])
print(mvs_cmd)

In [ ]:
os.system(mvs_cmd)

## Texture Mesh

In [ ]:
mvs_cmd = ' '.join([
    os.path.join(mvs_path, 'TextureMesh'),
    '--export-type', 'obj',
    '--working-folder', mvs_ws_path,
    '--input-file', os.path.join(mvs_ws_path, 'model/model_dense_mesh_refine.mvs'),
    '--output-file', os.path.join(mvs_ws_path, 'model/model.obj')
])
print(mvs_cmd)

In [ ]:
os.system(mvs_cmd)